In [1]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *

In [2]:
arcpy.env.overwriteOutput = True
# To run the script, the following datasets must be available in the geodatabase set below:
#   MyDEM - a digital elevation model for "your" watershed with spatial resolution 1m x 1m
#   rBuildings - a raster representation for buildings in "your" watershed area 
#   rWaterbodies - a raster representation for lakes (but not rivers) for "your" watershed area

In [4]:
#workspace
env.workspace = (r"\\Laptop-41p6i7do\uep\GIS\Modules\GIS for Climate Change\LM3\LM3 Bluespots/LM3 Bluespots.gdb")

In [5]:
print ("Fill all sinks")
arcpy.gp.Fill_sa("MyDEM", "dem_asf", "")

Fill all sinks


<geoprocessing server result object object at 0x000001FA5B130D50>

In [6]:
print ("Fill only small sinks")
arcpy.gp.Fill_sa("MyDEM", "dem_ssf", "0.2")

Fill only small sinks


<geoprocessing server result object object at 0x000001FA5AFF75D0>

In [7]:
print ("Minus ...")
arcpy.gp.Minus_sa("dem_asf", "dem_ssf", "Bluespot_CbC")

Minus ...


<geoprocessing server result object object at 0x000001FA5B130300>

In [8]:
# Part_B - Exclude Buildings and Lakes
print ("Con ...")
## syntax for the Con tool: input, true, output, false, condition
arcpy.gp.Con_sa("Bluespot_CbC", "1", "Bluespot_Pre", "0", "Value > 0")

Con ...


<geoprocessing server result object object at 0x000001FA5B130D50>

In [9]:
print ("Exclude buildings from blue spots ...")
arcpy.env.extent = "MyDEM"
arcpy.ddd.Reclassify("rBuildings", "Value", "1 1;NODATA 0", "rBuildings2", "DATA")
arcpy.gp.Minus_sa("Bluespot_Pre", "rBuildings2", "Bluespot_NoBld")
arcpy.gp.Reclassify_sa("Bluespot_NoBld", "Value", "1 1;0 0;-1 0", "Bluespot_NoBld2", "DATA")

Exclude buildings from blue spots ...


<geoprocessing server result object object at 0x000001FA5B130F30>

In [10]:
print ("Exclude water bodies from blue spots")
arcpy.ddd.Reclassify("rWaterbodies", "Value", "1 1;NODATA 0", "rWaterbodies2", "DATA")
arcpy.gp.Minus_sa("Bluespot_NoBld2", "rWaterbodies2", "Bluespot_NoBRC")
arcpy.gp.Reclassify_sa("Bluespot_NoBRC", "Value", "1 1;0 NODATA; -1 NODATA", "Bluespot_NoBRC2", "DATA")
arcpy.gp.RegionGroup_sa("Bluespot_NoBRC2", "BSRegions", "EIGHT", "WITHIN", "ADD_LINK", "0")

Exclude water bodies from blue spots


<geoprocessing server result object object at 0x000001FA5B195C00>

In [11]:
# Part_c_ Calculate Volume of Each Bluespot
print ("Zonal Statistics as Table - depths")
arcpy.AddField_management("BSRegions", "Volume", "FLOAT", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
arcpy.gp.ZonalStatisticsAsTable_sa("BSRegions", "VALUE", "Bluespot_CbC", "Sum_Depths", "DATA", "SUM")
arcpy.JoinField_management("BSRegions", "Value", "Sum_Depths", "Value", "SUM")
arcpy.management.CalculateField("BSRegions", "Volume", "!SUM!", "PYTHON3", "", "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.DeleteField("BSRegions", "SUM", "DELETE_FIELDS")

Zonal Statistics as Table - depths


<Result 'BSRegions'>

In [12]:
print ("Flow Direction")
arcpy.gp.FlowDirection_sa("dem_asf", "FlowDir_asf", "NORMAL", "", "D8")

Flow Direction


<geoprocessing server result object object at 0x000001FA5B1302D0>

In [13]:
print ("Watershed")
arcpy.gp.Watershed_sa("FlowDir_asf", "BSRegions", "Watersheds_asf", "VALUE")

Watershed


<geoprocessing server result object object at 0x000001FA44D98090>

In [14]:
print ("Build Raster Attribute Table")
arcpy.BuildRasterAttributeTable_management("Watersheds_asf", "NONE")
print ("Add Field (2)")
arcpy.AddField_management("Watersheds_asf", "WatershedArea", "FLOAT", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
print ("Calculate Field (2)")
arcpy.management.CalculateField("Watersheds_asf", "WatershedArea", "!Count!", "PYTHON3", "", "TEXT", "NO_ENFORCE_DOMAINS")

Build Raster Attribute Table
Add Field (2)
Calculate Field (2)


<Result 'Watersheds_asf'>

In [15]:
# Field Management
print ("Join Field (3)")
arcpy.JoinField_management("BSRegions", "Value", "Watersheds_asf", "Value", "WatershedArea")
print ("Add Field (3)")
arcpy.AddField_management("BSRegions", "FillUp", "FLOAT", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
print ("Calculate Field (3)")
##The Volume field contains the volume of each bluespot (in cubic meters).
##The FillUp field contains the amount of rainfall (in millimeters) needed to fill each bluespot in a cloudburst.
arcpy.management.CalculateField("BSRegions", "FillUp", "(!Volume! * 1000) / !WatershedArea!", "PYTHON3", "", "TEXT", "NO_ENFORCE_DOMAINS")

Join Field (3)
Add Field (3)
Calculate Field (3)


<Result 'BSRegions'>

In [16]:
# Transform bluespots to polygon
print ("Raster to Polygon (2)")
arcpy.RasterToPolygon_conversion("BSRegions", "fcBSRegions", "NO_SIMPLIFY", "VALUE", "SINGLE_OUTER_PART", "")
print ("Dissolve")
arcpy.Dissolve_management("fcBSRegions", "fcBSRegions2", "gridcode", "", "MULTI_PART", "DISSOLVE_LINES")
print ("Join Field (4)")
arcpy.JoinField_management("fcBSRegions2", "gridcode", "BSRegions", "Value", "Volume;FillUp")

Raster to Polygon (2)
Dissolve
Join Field (4)


<Result 'fcBSRegions2'>

In [17]:
print ("Add field for fillup values adjusted for 20 mm sewer capacity")
#### FillUp + 20 mmm assuming that the sewer system capacity can handle 20 mm a rain per hour
arcpy.AddField_management("fcBSRegions2", "FillUp20", "FLOAT", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
print ("Calculate Field (3)")
arcpy.CalculateField_management("fcBSRegions2", "FillUp20", "!FillUp! + 20", "PYTHON3", "")

Add field for fillup values adjusted for 20 mm sewer capacity
Calculate Field (3)


<Result 'fcBSRegions2'>